# Run SPARQL queries on a local RDF file

We will use the Java RDF library [Apache Jena](https://jena.apache.org/). Follow these instructions to install the library:

1. Copy the `apache-jena-3.16.0` folder from Lab3 to the same directory as the notebook, or download the [apache-jena-3.16.0.zip](http://apache.mirror.triple-it.nl/jena/binaries/apache-jena-3.16.0.zip) file and unzip it
2. Run the next cell (it will import all jars in the `apache-jena-3.16.0/lib` folder)

In [11]:
%jars /opt/apache-jena-*/lib/*.jar

In [67]:
import org.apache.jena.riot.RDFDataMgr;
import org.apache.jena.rdf.model.*;
import org.apache.jena.util.PrintUtil;
import org.apache.jena.vocabulary.RDF;
import org.apache.jena.query.Query;
import org.apache.jena.query.QueryExecution;
import org.apache.jena.query.QueryExecutionFactory;
import org.apache.jena.query.QueryFactory;
import org.apache.jena.query.QuerySolution;
import org.apache.jena.query.ResultSetFormatter;
import org.apache.jena.query.ResultSet;
import java.io.File;
import org.apache.commons.io.FileUtils;
import java.nio.charset.StandardCharsets;

## Load the RDF file in a Jena Model

Load the RDF file, and create `runSparqlQuery()` function to run SPARQL queries in a single call from a string or a file.

In [68]:
Model model = RDFDataMgr.loadModel("family.ttl");

In [69]:
static void runSparqlQuery(String queryString, Model model) throws java.io.IOException {
    if (queryString.endsWith(".rq")) {
        queryString = FileUtils.readFileToString(new File(queryString), StandardCharsets.UTF_8);
    }
    System.out.println(queryString);
    Query query = QueryFactory.create(queryString);
    QueryExecution qexec = QueryExecutionFactory.create(query, model);
    ResultSetFormatter.out(qexec.execSelect(), model);
} 

## SPARQL query from string

Java does not support multiline strings.

In [70]:
String queryString = "PREFIX schema: <https://schema.org/> \n" +
    "SELECT * WHERE{ \n" + 
    "?entity a ?type .\n" +
    "}";

In [71]:
runSparqlQuery(queryString, model)

----------------------------------
| entity         | type          |
| family:Miranda | schema:Person |
| family:Pierre  | schema:Person |
----------------------------------


## SPARQL query from a file

Provide the SPARQL query in a file **with the extension `.rq`**

In [72]:
runSparqlQuery("query_get_type.rq", model)

PREFIX schema: <https://schema.org/>
PREFIX family: <https://my-family.org/>
SELECT * 
WHERE {
#     ?entity a ?type .
    ?entity a schema:Person ;
              schema:parent ?mother .
}
-------------------
| entity | mother |
-------------------


# Use Inference Model

In [73]:
//InfModel rdfsModel = ModelFactory.createRDFSModel(RDFDataMgr.loadModel("Lab3_RDFS_graph.ttl"));
InfModel rdfsModel = ModelFactory.createRDFSModel(RDFDataMgr.loadModel("family.ttl"));

In [74]:
runSparqlQuery("query_get_type.rq", rdfsModel)

PREFIX schema: <https://schema.org/>
PREFIX family: <https://my-family.org/>
SELECT * 
WHERE {
#     ?entity a ?type .
    ?entity a schema:Person ;
              schema:parent ?mother .
}
------------------------------------
| entity         | mother          |
| family:Miranda | family:Mathilde |
| family:Miranda | family:Pierre   |
------------------------------------
